In [9]:
# Initial imports
import os
import json
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

In [10]:
# Load .env enviroment variables
load_dotenv()

True

In [14]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

In [38]:
# Format current date as ISO format
start_date = pd.Timestamp("2015-12-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-04-13", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["TSLA", "SPY"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG
df_ticker = api.get_bars(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df


# Reorganize the DataFrame
# Separate ticker data
TSLA = df_ticker[df_ticker['symbol']=='TSLA'].drop('symbol', axis=1)
SPY = df_ticker[df_ticker['symbol']=='SPY'].drop('symbol', axis=1)

In [39]:
TSLA_close = TSLA[['close']]

In [40]:
TSLA_close['daily_returns'] = TSLA_close['close'].pct_change()

<ipython-input-40-3176783d4060>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TSLA_close['Daily Returns'] = TSLA_close['close'].pct_change()
